## Индивидуальное задание

In [1]:
import warnings
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Загрузка данных о сердечных заболеваниях в Швейцарии
    data = pd.read_csv("./heartDisease/processed.switzerland.csv", delimiter=',')

    # Разделение данных на целевую переменную и данные для обучения
    X = data.drop("num", axis=1)
    y = data["num"]

    # DataFrame для результатов измерения точности предсказывания
    results_df = pd.DataFrame(columns=['test_size', 'k', 'accuracy', 'cv_accuracy']) 

    # перебор k значений от 1 до 31 и долей тестовых данных от [0.1 до 0.5]
    k_values = range(1, 31)
    for test_size in [0.1, 0.2, 0.3, 0.4, 0.5]:
        for k in k_values:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
            # обучнеие и измерение точности
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_predicted = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_predicted)
            scores = cross_val_score(knn, X, y)
            results_df = pd.concat([
                results_df,
                pd.DataFrame({
                    'test_size': test_size,
                    'k': [k],
                    'accuracy': [accuracy],
                    'cv_accuracy': scores.mean()
                })],
                ignore_index=True
            )

    results_df = results_df.sort_values(by='accuracy', ascending=False)
    print(results_df)

     test_size   k  accuracy  cv_accuracy
116        0.4  27  0.460000     0.406667
118        0.4  29  0.440000     0.390667
119        0.4  30  0.440000     0.374000
137        0.5  18  0.435484     0.398000
65         0.3   6  0.432432     0.365667
..         ...  ..       ...          ...
27         0.1  28  0.153846     0.358000
1          0.1   2  0.076923     0.260333
3          0.1   4  0.000000     0.309667
4          0.1   5  0.000000     0.318000
2          0.1   3  0.000000     0.301000

[150 rows x 4 columns]


Таким образом самый точный результат выдаёт размер тестового набора 40% и k=27